# TP 5-6 : Partie 2 : Problème avec précédences et ressources disjonctives
***

## Job-shop : ordonnancement avec contraintes de précédence et contraintes de ressources
***

Le problème du job-shop consiste à planifier un ensemble de travaux pour minimiser la durée
totale d’exécution tout en respectant des contraintes de précédence (chaque travail est décomposé en
opérations à réaliser dans l’ordre) et de ressources (chaque opération utilise une machine et chaque
machine ne peut traiter qu’une opération à la fois).

<img src="Tableau2.png" alt="Tableau2"/>

### 2.1 Relaxation des contraintes de ressources

Soit une relaxation $(R)$ qui consiste à ignorer les contraintes de ressources du job-shop. Et puisque le *Tableau1* dépend juste de la durée d'une tâche, donc $(R)$ est équivalent au problème $(P1)$.

##### Modélisation avec la méthode du graphe potentiel-tâche

In [164]:
# Initialisation
import Pkg; Pkg.add("Clp")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [165]:
function getIndex(vertex, n, item)
    for i in 1 : n
        if vertex[i] == item
            return i
        end
    end
end

function bellman_ford(n, m, graph, vertex, source)
    
    distance = zeros(n)
    
    for i in 1 : n
        distance[i] = typemax(Int64)
        previous[i] = "debut"
    end
    
    distance[source[1]] = 0
    distance[source[2]] = 0
    
    for i in 1 : n
        for j in 1 : m
            u = graph[j][1]
            v = graph[j][2]
            w = -graph[j][3]

            if (distance[getIndex(vertex, n, u)] + w < distance[getIndex(vertex, n, v)])
                distance[getIndex(vertex, n, v)] = distance[getIndex(vertex, n, u)] + w
                previous[getIndex(vertex, n, v)] = u
            end
        end
    end
    
    for i in 1 : m
        u = graph[i][1]
        v = graph[i][2]
        w = -graph[i][3]
        if (distance[getIndex(vertex, n, u)] != typemax(Int64) 
            && distance[getIndex(vertex, n, u)] + w  < distance[getIndex(vertex, n, v)])  
            println("Graph contains negative weight cycle")
        end
    end

    println("t = ", abs.(distance)[2 : (n - 1)])
    println("tfin = ", abs.(distance)[n])
end

bellman_ford (generic function with 1 method)

#### Test avec l'exemple du sujet

In [166]:
vertex = ["1.1", "1.2", "1.3", "2.1", "2.2", "2.3", "fin"]
previous = ["1.1", "1.1", "1.1", "1.1", "1.1", "1.1", "1.1"]
source = [1, 4]
graph = [
    ("1.1", "1.2", 6),
    ("1.2", "fin", 7),
    ("2.1", "2.2", 3),
    ("2.2", "2.3", 5),
    ("2.3", "fin", 1)
    ]
bellman_ford(length(vertex), length(graph), graph, vertex, source)

t = [6.0, 9.223372036854776e18, 0.0, 3.0, 8.0]
tfin = 13.0


#### Modélisation avec la programmation linéaire

In [167]:
using JuMP
using Clp

# Data
D = [6 7 0; 3 5 1]
NbTr = 2
NbOp = 3

model = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model, t[1:NbTr, 1:NbOp] >= 0)
@variable(model, tfin)

# define objective function
@objective(model, Min, tfin)

# define constraints: t_j - t_i  >= D[i], \forall i predecesseur de j
@constraint(model, t[1,2] - t[1,1] >= D[1,1] )
@constraint(model, t[1,3] - t[1,2] >= D[1,2] )
@constraint(model, t[2,2] - t[2,1] >= D[2,1] )
@constraint(model, t[2,3] - t[2,2] >= D[2,2] )


#define constraints: tfin - t_i >= Duree[i], \forall i predecesseur de j
@constraint(model, tfin - t[2,3] >= D[2,3] )
@constraint(model, tfin - t[1,3] >= D[1,3] )

println(model)

print("start solve ... ")
optimize!(model)
print("... end solve")


println("\n\nSolution PL:\n \t t=", value.(t), "\t tfin=", value(tfin))

Min tfin
Subject to
 -t[1,1] + t[1,2] ≥ 6.0
 -t[1,2] + t[1,3] ≥ 7.0
 -t[2,1] + t[2,2] ≥ 3.0
 -t[2,2] + t[2,3] ≥ 5.0
 -t[2,3] + tfin ≥ 1.0
 -t[1,3] + tfin ≥ 0.0
 t[1,1] ≥ 0.0
 t[2,1] ≥ 0.0
 t[1,2] ≥ 0.0
 t[2,2] ≥ 0.0
 t[1,3] ≥ 0.0
 t[2,3] ≥ 0.0

start solve ... ... end solve

Solution PL:
 	 t=[0.0 6.0 13.0; 0.0 3.0 8.0]	 tfin=13.0


### 2.2 Modélisation des contraintes de ressources

Principe : Si deux opérations $ij$ et $kl$ nécessitent une même ressource, alors il faut imposer que
l’une se termine avant que l’autre ne commence. L’ordre n’étant pas connu à l’avance, il faut donc
réussir à modéliser la contrainte **“$t_{ij} − t_{kl} ≥ d_{kl}$ où $t_{kl} − t_{ij} ≥ d_{ij}$”** qui n’est pas linéaire.

#### 2.2.1  Méthode du bigM

L’idée est d’utiliser une variable binaire $x_{kl\_ij}$ qui vaudra $1$ si kl est exécuté avant $ij$ et $0$ sinon. Ce modèle n’est pas purement linéaire à cause des contraintes $(15)$.

<img src="Figure2.png" alt="Figure2"/>

#### 2.2.2 Méthode des graphes disjonctifs

L’idée est d’utiliser un graphe **G(X, U, D)** appelé graphe disjonctif, qui est un outil pratique de
modélisation pour ressources non partageables où :
<ul>
    <li>L’ensemble des sommets est <strong>X</strong></li>
    <li>L’ensemble des arcs est composé de :
        <ul>
            <li><strong>U</strong> : partie conjonctive représentant les gammes opératoires d’un travail
                <ul><li>∀(ij, ik) ∈ U, tik − tij ≥ pij , représenté par un arc orienté de ij vers ik</li></ul>
            </li>
            <li><strong>D</strong> : partie disjonctive associée aux conflits d’utilisation d’une ressource non partageable
                <ul><li>∀(ij, kl) ∈ D, on a soit tkl − tij ≥ pij , soit tij − tkl ≥ pkl, représenté par une arête ou paire de disjonction</li></ul>
        </ul>
    </li>
</ul>
Le graphe disjonctif correspondant aux données des Tableaux 2(a) et 2(b) est fourni ci-dessous.
Les arêtes en pointillés à double tête y représentent les paires de disjonction.

<img src="graphe_disjonctif.png" alt="graphe_disjonctif"/>

### 2.3 Résolution

Implémentation d'une **PSE** pour résoudre le problème du job-shop sur les données du *Tableau 2*, celles du *Tableau 3*.

<img src="Tableau3.png" alt="Tableau3"/>


#### 2.3.1 PSE basée sur la relaxation linéaire du modèle avec bigM 

Principe : utiliser le solveur linéaire Clp pour résoudre les relaxations linéaires Paramètres proposés.
<ul>
    <li>
        Critère de séparation
        <ul>
            <li>une variable fractionnaire</li>
        </ul>
    </li>
    <li>
        Tests de sondabilité
        <ul>
            <li>TA : réussi si la relaxation linéaire (RL) n’a pas de solution admissible</li>
            <li>TO : réussi si la solution de la RL est pire que la meilleure solution connue</li>
            <li>TR : réussi si la solution de la RL est entière</li>
        </ul>
    </li>
    <li>Stratégie d’exploration : priorité à la profondeur</li>
</ul>

##### Récupération des données

In [168]:
MachinesParJob = [1 3 0; 1 2 3]
Duree = [6 7 0; 3 5 1]
bigM = sum(Duree)

22

##### Procédure d'application des tests de sondabilités TA, TO et TR pour le cas de la relaxation linéaire

In [169]:
function TestsSondabilite_relaxlin(model2, varsbin, BestTfin, Bestsol)
    TA, TO, TR = false, false, false
    if(termination_status(model2) == MOI.INFEASIBLE) # Test de faisabilite
        TA = true
        println("TA")
    elseif(objective_value(model2) >= BestTfin) # Test d'optimalite
        TO = true
        println("TO")
    elseif( prod(abs.([round.(v, digits = 0) for v in value.(varsbin)] - value.(varsbin)) .<= fill(10 ^ -5, size(varsbin))) 
        ) # Test de resolution
        TR = true
        println("TR")
        if (value(tfin) <= BestTfin)
            Bestsol = value.(t)
            BestTfin = value(tfin)
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestTfin
end

TestsSondabilite_relaxlin (generic function with 1 method)

##### Procédure de séparation et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [170]:
function SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    # le noeud est non-sondable. Appliquer le critère de séparation pour le séparer en sous-noeuds 
    # et choisir un noeud-fils le plus à gauche   
    
    # find a fractionnal variable
    i, var = 1, 0
    while((i <= length(varsshouldbebinary)) && (var == 0))
        #if(varsshouldbebinary[i] ∉ listvars)
        if(abs(round(value(varsshouldbebinary[i]), digits = 0) - value(varsshouldbebinary[i]) ) >= 10 ^ -5)
            var = varsshouldbebinary[i]
        end
        i += 1
    end
    
    #=
    #find most fractionnal variable
    i, var, maxfrac = -1, 0, 0.0
    for i in 1:length(varsshouldbebinary)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= maxfrac) 
            #if a variable is more fractinonal
            var=varsshouldbebinary[i]
            maxfrac=abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) )
            #println(i, " ", var, " ", maxfrac)
        end
    end
    =#
    

    set_lower_bound(var, 1.0)
    set_upper_bound(var, 1.0)

    push!(listvars, var) # stocker l'identite de la variable choisie pour la séparation
    push!(listvals, 1.0) # stocker la branche choisie, identifiee par la valeur de la variable choisie
    listvars, listvals
end


function ExplorerAutreNoeud_relaxlin(listvars, listvals)
    # this node is sondable, go back to parent node then right child if possible
    
    stop = false
    # go back to parent node
    var = pop!(listvars)
    theval = pop!(listvals)
    set_lower_bound(var, 0.0)
    set_upper_bound(var, 1.0)

    # go to right child if possible, otherwise go back to parent
    while( (theval == 0.0) && (length(listvars) >= 1))
        var = pop!(listvars)
        theval = pop!(listvals)
        set_lower_bound(var, 0.0) 
        set_upper_bound(var, 1.0)
    end
    if theval == 1.0
        set_lower_bound(var, 0.0)
        set_upper_bound(var, 0.0)
        push!(listvars, var)
        push!(listvals, 0.0)
    else
        println("\nFINISHED")
        stop = true
    end
    listvars, listvals, stop 
end

ExplorerAutreNoeud_relaxlin (generic function with 1 method)

##### Creation de la relaxation linéaire (= modèle associé au noeud 0)

In [171]:
# ROOT NODE
using JuMP
using Clp

model2 = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model2, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model2, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model2, t[i in 1:2, j in 1:3] >= 0)
@variable(model2, tfin)

# define objective function
@objective(model2, Min, tfin)

# define constraints: t_i(j+1) - t_ij  >= Duree[i,j], \forall i,j
@constraint(model2, t[1,2] - t[1,1] >= Duree[1,1] )
@constraint(model2, t[1,3] - t[1,2] >= Duree[1,2] )
@constraint(model2, t[2,2] - t[2,1] >= Duree[2,1] )
@constraint(model2, t[2,3] - t[2,2] >= Duree[2,2] )

#define constraints: tfin - t_ij >= Duree[i,j], \forall ij
@constraint(model2, tfin - t[1,3] >= Duree[1,3] )
@constraint(model2, tfin - t[2,3] >= Duree[2,3] )


# define x variables as CONTINUOUS (recall that it is not possible to define binary variables in Clp)
@variable(model2, 0 <= x_2_1__1_1 <= 1)
@variable(model2, 0 <= x_2_3__1_2 <= 1)
varsshouldbebinary=[x_2_1__1_1,x_2_3__1_2]


# define bigM constraints linking x and t variables
@constraint(model2, t[2,1] - t[1,1] >=  Duree[1,1] - bigM*x_2_1__1_1)
@constraint(model2, t[1,1] - t[2,1] >=  Duree[2,1] - bigM*(1-x_2_1__1_1))
@constraint(model2, t[2,3] - t[1,2] >=  Duree[1,2] - bigM*x_2_3__1_2)
@constraint(model2, t[1,2] - t[2,3] >=  Duree[2,3] - bigM*(1-x_2_3__1_2))


println(model2)

Min tfin
Subject to
 -t[1,1] + t[1,2] ≥ 6.0
 -t[1,2] + t[1,3] ≥ 7.0
 -t[2,1] + t[2,2] ≥ 3.0
 -t[2,2] + t[2,3] ≥ 5.0
 -t[1,3] + tfin ≥ 0.0
 -t[2,3] + tfin ≥ 1.0
 -t[1,1] + t[2,1] + 22 x_2_1__1_1 ≥ 6.0
 t[1,1] - t[2,1] - 22 x_2_1__1_1 ≥ -19.0
 -t[1,2] + t[2,3] + 22 x_2_3__1_2 ≥ 7.0
 t[1,2] - t[2,3] - 22 x_2_3__1_2 ≥ -21.0
 t[1,1] ≥ 0.0
 t[2,1] ≥ 0.0
 t[1,2] ≥ 0.0
 t[2,2] ≥ 0.0
 t[1,3] ≥ 0.0
 t[2,3] ≥ 0.0
 x_2_1__1_1 ≥ 0.0
 x_2_3__1_2 ≥ 0.0
 x_2_1__1_1 ≤ 1.0
 x_2_3__1_2 ≤ 1.0



##### Boucle principale : résoudre la relaxation linéaire, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [172]:

listvars=[]
listvals=[]

BestTfin = bigM
Bestsol = []

current_node_number = 0
stop = false

while(!stop)
    
    println("\nNode number ", current_node_number, ": \n-----\n", model2)

    print("Solve : start ... ")
    status = optimize!(model2)
    println("... end")

    println("\nSolution relax lin",); [print("\t", name(v),"=",value(v)) for v in all_variables(model2)]; 
    println(" "); println("\nSolution precedemment memorisee ", Bestsol, " avec date de fin ", BestTfin, "\n")

    TA, TO, TR, Bestsol, BestTfin = TestsSondabilite_relaxlin(model2, varsshouldbebinary, BestTfin, Bestsol)

    is_node_sondable = TA || TO || TR
    
    if(!is_node_sondable)
        listvars, listvals = SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    else
        listvars, listvals, stop = ExplorerAutreNoeud_relaxlin(listvars, listvals)
    end

    current_node_number = current_node_number + 1
end

println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)



Node number 0: 
-----
Min tfin
Subject to
 -t[1,1] + t[1,2] ≥ 6.0
 -t[1,2] + t[1,3] ≥ 7.0
 -t[2,1] + t[2,2] ≥ 3.0
 -t[2,2] + t[2,3] ≥ 5.0
 -t[1,3] + tfin ≥ 0.0
 -t[2,3] + tfin ≥ 1.0
 -t[1,1] + t[2,1] + 22 x_2_1__1_1 ≥ 6.0
 t[1,1] - t[2,1] - 22 x_2_1__1_1 ≥ -19.0
 -t[1,2] + t[2,3] + 22 x_2_3__1_2 ≥ 7.0
 t[1,2] - t[2,3] - 22 x_2_3__1_2 ≥ -21.0
 t[1,1] ≥ 0.0
 t[2,1] ≥ 0.0
 t[1,2] ≥ 0.0
 t[2,2] ≥ 0.0
 t[1,3] ≥ 0.0
 t[2,3] ≥ 0.0
 x_2_1__1_1 ≥ 0.0
 x_2_3__1_2 ≥ 0.0
 x_2_1__1_1 ≤ 1.0
 x_2_3__1_2 ≤ 1.0

Solve : start ... ... end

Solution relax lin
	t[1,1]=0.0	t[2,1]=0.0	t[1,2]=6.0	t[2,2]=3.0	t[1,3]=13.0	t[2,3]=8.0	tfin=13.0	x_2_1__1_1=0.27272727272727276	x_2_3__1_2=0.22727272727272732 

Solution precedemment memorisee Any[] avec date de fin 22

non sondable

Node number 1: 
-----
Min tfin
Subject to
 -t[1,1] + t[1,2] ≥ 6.0
 -t[1,2] + t[1,3] ≥ 7.0
 -t[2,1] + t[2,2] ≥ 3.0
 -t[2,2] + t[2,3] ≥ 5.0
 -t[1,3] + tfin ≥ 0.0
 -t[2,3] + tfin ≥ 1.0
 -t[1,1] + t[2,1] + 22 x_2_1__1_1 ≥ 6.0
 t[1,1] - t[2,1

In [173]:
println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)


******

Optimal value = 15.0

Optimal t=[0.0 6.0 15.0; 6.0 9.0 14.0]


#### 2.3.2 PSE basée sur le graphe disjonctif

Principe : utiliser votre algorithme de calcul de plus long chemin pour résoudre les relaxations du graphe disjonctif.
Paramètres proposés:
<ul>
    <li>
        Principe de séparation
        <ul>
            <li>Choix d’un arc dans une paire de disjonction</li>
        </ul>
    </li>
    <li>
        Fonction d’évaluation (borne inférieure)
        <ul>
            <li>
                Recherche du chemin critique dans un graphe possédant (1) tous les arcs conjonctifs et
(2) les arcs disjonctifs des conflits déjà arbitrés
                <ul>
                    <li>
                        En déduire les tests TA, TO, TR résultants
                    </li>
                </ul>
            </li>
        </ul>
    </li>
    <li>Stratégie d’exploration : priorité à la profondeur</li>
</ul>

##### Modélisation du problème

In [174]:
function getIndex(vertex, n, item)
    for i in 1 : n
        if vertex[i] == item
            return i
        end
    end
end

function bellman_ford_v1(n, m, graph, graph_disj, vertex, source)
    
    distance = zeros(n)
    
    for i in 1 : n
        distance[i] = -Inf
        previous[i] = "debut"
    end
    distance[1] = 0
    distance[source[1]] = 0
    distance[source[2]] = 0
    p = 0
    
    for i in 1 : n
        
        for j in 1 : m
            
            u = graph[j][1]
            v = graph[j][2]
            w = graph[j][3]
            
            if ((distance[getIndex(vertex, n, u)] + w > distance[getIndex(vertex, n, v)])
                || (getIndex(vertex, n, v) in source && i == 1))
                
                condition = false
                
                for k in 1 : size(graph_dis)[1]
                    
                    u1 = graph_dis[k][1]
                    v1 = graph_dis[k][2]
                    w1 = graph_dis[k][3]    
                    
                    if v == v1
                        
                        for l in 1 : size(graph_dis)[1]
                            u2 = graph_dis[l][1]
                            if u2 == v1
                                p = l
                            end
                        end
                        
                        condition = true
                        
                        if distance[getIndex(vertex, n, v)] + graph_dis[p][3] < distance[getIndex(vertex, n, u1)] + 
                                w1
                            distance[getIndex(vertex, n, v)] = min(distance[getIndex(vertex, n, u)] + w + w1,
                                distance[getIndex(vertex, n, u1)] + w1)
                        else 
                            distance[getIndex(vertex, n, v)] = distance[getIndex(vertex, n, u)] + w
                        end
                        
                    end
                    
                end
                
                if !condition
                    distance[getIndex(vertex, n, v)] = distance[getIndex(vertex, n, u)] + w
                end
            end
        end
    end
    
    println("t = ", distance[1 : (n - 1)])
    println("tfin = ", distance[n])

end

bellman_ford_v1 (generic function with 1 method)

##### Test avec l'exemple du sujet

In [175]:
vertex = ["debut", "1.1", "1.2", "1.3", "2.1", "2.2", "2.3", "fin"]
previous = ["debut", "debut", "debut", "debut", "debut", "debut", "debut", "debut"]
source = [2, 5]

graph = [
    ("debut", "1.1", 0), ("debut", "2.1", 0),
    ("1.1", "1.2", 6),
    ("1.2", "fin", 7),
    ("2.1", "2.2", 3),
    ("2.2", "2.3", 5),
    ("2.3", "fin", 1),
    ]          

graph_dis = [
    ("1.1", "2.1", 6),
    ("1.2", "2.3", 7),
    ("2.1", "1.1", 3),
    ("2.3", "1.2", 1),
    ]

bellman_ford_v1(length(vertex), length(graph), graph, graph_dis, vertex, source)

t = [0.0, 0.0, 6.0, -Inf, 6.0, 9.0, 14.0]
tfin = 15.0
